**Note:** 이 `notebook`은 `python 2.7`을 사용하기 위해 만들어졌습니다. 하지만 `%%bash`를 각 셀의 시작에 작성하게되면 `shell` 명령어도 사용할 수 있습니다. 

In [ ]:
import utils.feature_viz.feature_viz as fv
import plotly.graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected=True)
import numpy as np

# 2.5: Examining the `MFCC`s

`INSTRUCTIONAL` 디렉토리에는 다음의 [third-party repository](https://github.com/vesis84/kaldi-io-for-python)의 복사본이 `utils/feature_viz`에 저장되어 있습니다.  `utils/feature_viz/kaldi_io.py`는 `.ark` 파일을 `python`에서 읽어올 수 있게 도와줍니다. 

In [ ]:
%%bash
ls utils/feature_viz/kaldi_io_for_python/

또한 `feature_viz.py`라는 `python` 모듈을 사용하여서 추출한 `MFCC` 특징들을 확인해보도록 하겠습니다. 

## Choosing an audio sample

추출된 `mfcc` 값들은 `mfcc` 디렉토리에 위치합니다. 

In [ ]:
%%bash
ls mfcc | grep mfcc

파일의 갯수는 우리가 병렬처리에 사용한 thread의 수와 같다는 것을 알 수 있습니다. 그리고 각각의 `int` 파일에 상응하는 `.ark`와 `.scp` 파일이 있습니다. 

**Note:** 만약 *특정한* 음성 파일을 확인하고 싶을 경우, 어떠한 `.ark` 파일에 해당 음성이 들어있는지 확인하여야 합니다. 실제로 음성 파일을 검색하여 내용을 확인해보도록 하겠습니다. 

In [ ]:
%%bash
head raw_data/librispeech-transcripts.txt 

먼저 어떤 `.ark` 파일에 해당 내용이 있는지 확인하여야 합니다. 

In [ ]:
%%bash 
for f in `ls mfcc/raw_mfcc_*.scp`; do
    match=$(cat $f | grep "1272-128104-0000")
    if [[ -z $match ]]; then
        echo "not found in $f"
    else
        echo "found in $f: $match"
    fi
done

## Reading in features

In [ ]:
SAMPLE = "1272-128104-0000"
ARK_SAMPLE = "raw_mfcc_train_dir.1.scp"

`read_in_features()` will read in the features for *all* of the utterances found in our `.ark`

`read_in_features()`는 `.ark` 파일에 있는 모든 utterances의 `mfcc` 값을 읽어옵니다. 

In [ ]:
feats_in = fv.read_in_features("mfcc/{}".format(ARK_SAMPLE))
list(feats_in.keys())[:10]

파일 형식을 자세히 살펴보면, 각각의 `값`은 `numpy array`의 구조로 해당 프레임의 `mfcc` 값을 표현하고 있다는 것을 알 수 있습니다. 

In [ ]:
feats_in[SAMPLE]

## Understanding the feature shape

`array`의 구조는 `(프레임의 수 x mfcc 계수의 수)` 입니다. 

In [ ]:
feats_in[SAMPLE].shape

`feature_viz.py`에는 프레임의 수와 feature의 수를 쉽게 출력할 수 있는 기능이 있습니다. 

In [ ]:
fv.get_num_frames(feats_in[SAMPLE]), fv.get_num_features(feats_in[SAMPLE])

`MFCC` feature를 추출하기 위해 사용한 `mfcc` configuration 파일을 확인해보면, 왜 array가 저런 구조를 가지게 되었는지 확인할 수 있습니다. 

In [ ]:
%%bash
cat conf/mfcc_defaults.conf

`--num-ceps`는 몇 개의 `mfcc` 계수를 feature로 사용할지 정하는 파라미터입니다. 

프레임의 갯수는 `--frame-length`와 `--frame-shift`로 결정됩니다. (물론 실제 음성 파일의 길이도 프레임의 갯수에 영향을 끼칩니다.) `kaldi`에서 기본적으로 사용하는 파라미터는 `--frame-length`는 `25 ms`, 그리고 다음 프레임은 이전 프레임이 끝나는 시점에 이어서 시작하는 것이 아니라 이전 프레임에서 `10 ms` 이동한 시점에서 시작하여 `25 ms` 구간입니다. 그러므로 각각의 프레임은 겹치는 구간이 있습니다. 

"1272-128104-0003"은 전사 길이가 길기 때문에, feature 또한 많을 것이라고 추측할 수 있습니다. 

In [ ]:
cat raw_data/librispeech-transcripts.txt | grep -E "1272-128104-000[03]"

In [ ]:
fv.get_num_frames(feats_in["1272-128104-0003"])

하지만 실제로는 다른 음성 파일과 `feature`의 갯수는 동일합니다. 

In [ ]:
fv.get_num_features(feats_in["1272-128104-0003"])

## Viewing the features

`feature_viz.py` 파일은 `plot_frames()`라는 function을 가지고 있습니다. 이 function은 음성 파일의 `n`개의 프레임에서 얻은 `MFCC` 값을 그래프로 보여줍니다. 

**Note:** 이 `notebook`에서 바로 그래프를 보기 위해서, `py.iplot([output])` 기능을 사용합니다. `output`은 `plot_frames()`의 결과값입니다. 

In [ ]:
# the one *required* argument is a numpy array of features for any number of frames
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][60:61]   # [x:y] will return the x^th frame
    )
)

위 셀을 실행하면 음성 파일에서 첫 번째 프레임의 feature를 확인할 수 있습니다. `mode=bar`라는 논항을 추가하면 `bar` 그래프로도 출력할 수 있습니다. `vector`는 `discrete`하다는 점에서 `bar` 그래프가 조금 더 직관적일 수 있습니다 (*e.g.,* `x=3.5`에 해당하는 값은 존재하지 않습니다. 

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][60:61],   # [x:y] will return the x^th frame
        mode='bar'
    )
)

`plot_frames()`는 또한 *연속된* 프레임의 값을 같은 그래프에 겹쳐서 표시할 수도 있습니다. 다음 그래프는 다섯 개의 연속된 프레임의 값을 출력한 그래프입니다. 

**Note:** `plotly`를 이용하면 범례에서 특정 선을 클릭함으로써 선을 투명하게 하거나 다시 나타나게 할 수 있습니다. 

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][60:68]
    )
)

위 그래프 또한 `bar` 그래프로 나타낼 수 있습니다. 

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][60:68],
        mode='bar'
    )
)

여러 프레임에 걸쳐서 *특정한* `mfcc` 값을 비교하는 것이 더 유용할 수 있습니다. 예를 들어, 첫 번째 *coefficient*는 모든 프레임에서 *양수*로 나타남을 확인할 수 있습니다. 

## Including `phone` information

`plot_frames()`를 잘 이용하면, 각각의 `프레임`에서 `예측된` phone의 정보를 이용하여 그래프를 그릴 수 있습니다. `예측된` phone이라고 이야기 하는 이유는 프레임을 `정렬`하여서 전사 파일과 맞추는 과정은 `ASR` pipeline에서 이루어지지만, 그 정확성은 pipeline의 질에 따라 바뀔 수 있기 때문입니다. 

우리는 정렬된 파일을 생성하는 과정을 아직까지 실행하지 않았습니다. (`ali.*.gz`라고 불리는 파일들이 존재하지 않습니다.) 그래서 시각화를 위해 필요한 자료를 `resource_files/feature_vis/all_ali`에 첨부하였습니다. 이 파일은 압축이 해제되어 있으며, training 데이터에 있는 모든 음성 파일의 `alignment` 정보를 담고 있습니다. 

In [ ]:
%%bash
ls resource_files/feature_viz

`C++` function인 `ali-to-phones`을 실행하여서 정렬된 내용을 확인할 수 있습니다. 

**Note:** 일반적으로 우리는 `source` `path.sh` (`. path.sh`)를 이용하여서 전체 경로를 입력하지 않고도 C++ function을 실행할 수 있었습니다. 하지만 현재 우리가 `python` `notebook`을 이용하고 있기 때문에 C++ function을 사용하기 위해서는 전체 경로를 입력하여야 합니다. 

In [ ]:
%%bash
${KALDI_PATH}/src/bin/ali-to-phones

해당 기능을 이용하기 위해서는 acoustic model이 필요합니다. 아직 acoustic model을 구축하지는 않았지만, 시각화를 위해서 해당 모델을 `resource_files/feature_viz/model_for_alignments.mdl`로 첨부하였습니다. 

In [ ]:
%%bash
# using ark,t:- we are telling the function to output to STDOUT
${KALDI_PATH}/src/bin/ali-to-phones \
    resource_files/feature_viz/model_for_alignments.mdl \
    ark:resource_files/feature_viz/all_ali \
    ark,t:- | grep "1272-128104-0000"

위 셀을 실행하면, 주어진 utterance를 phone으로 변환할 수 있는 index 값들이 생성됩니다. 만약 실제 phone을 확인하고 싶다면 위에서 생성된 결과를 `int2sym.pl`의 input으로 넣어주어야 합니다. 

In [ ]:
%%bash
${KALDI_INSTRUCTIONAL_PATH}/utils/int2sym.pl

이 방법을 이용하려면 `symtab`(symbol table)이 필요합니다. 우리는 이미 `data/lang/phones.txt` 파일에 해당 내용을 넣어두었습니다. 

In [ ]:
%%bash
head data/lang/phones.txt

In [ ]:
cat raw_data/librispeech-transcripts.txt | grep -E "1272-128104-0000"

In [ ]:
%%bash
${KALDI_PATH}/src/bin/ali-to-phones \
    resource_files/feature_viz/model_for_alignments.mdl \
    ark:resource_files/feature_viz/all_ali \
    ark,t:- |\
    ${KALDI_INSTRUCTIONAL_PATH}/utils/int2sym.pl -f 2- data/lang/phones.txt |\
    grep "1272-128104-0000"

결과물을 보았을 때, 우리가 음성 파일에 대해서 이미 알고 있는 정보에 비해 정교하지 못하다는 것을 확인할 수 있습니다. 이러한 경우에 `ali-to-phones` function은 "보여지는 것"에 중점을 두고 같은 phone이 연속되는 경우 하나로 합쳐버립니다. 실제로 하나의 phone이 하나의 프레임보다 긴 구간 동안 발화될 수 있기 때문에, 이는 당연하다고 할 수 있습니다. 

각 `프레임`에 해당하는 `phone`을 얻기 위해서 `--per-frame`이라는 논항을 추가하겠습니다. 

In [ ]:
%%bash
${KALDI_PATH}/src/bin/ali-to-phones \
    --per-frame=true \
    resource_files/feature_viz/model_for_alignments.mdl \
    ark:resource_files/feature_viz/all_ali \
    ark,t:- |\
    ${KALDI_INSTRUCTIONAL_PATH}/utils/int2sym.pl -f 2- data/lang/phones.txt |\
    grep "1272-128104-0000"

이제 training 데이터에 있는 각각의 음성 파일에서 우리가 얻고자 하는 정보를 얻을 수 있게 되었습니다. 아래 명령어를 통해서 해당 정보를 얻고, 그 결과를 `resource_files/feature_viz/all_ali_phoned`에 저장하겠습니다. 

In [ ]:
%%bash
${KALDI_PATH}/src/bin/ali-to-phones \
    --per-frame=true \
    resource_files/feature_viz/model_for_alignments.mdl \
    ark:resource_files/feature_viz/all_ali \
    ark,t:- |\
    ${KALDI_INSTRUCTIONAL_PATH}/utils/int2sym.pl -f 2- data/lang/phones.txt > resource_files/feature_viz/all_ali_phoned

In [ ]:
%%bash
cat resource_files/feature_viz/all_ali_phoned | grep "1272-128104-0000"

`feature_viz.py`는 `alignment` 파일을 읽어서 `python` `<dict>`의 형태로 저장하는 기능을 제공합니다. 그리고 `<dict>`의 `value`는 프레임의 갯수와 동일한 갯수의 phone을 담은 `<list>`입니다. 

In [ ]:
ali_in = fv.read_in_alignments("resource_files/feature_viz/all_ali_phoned")
ali_in[SAMPLE][60:68]

In [ ]:
len(ali_in[SAMPLE]) == fv.get_num_frames(feats_in[SAMPLE])

이제 `plot_frames()`에 적절한 `<list>`를 추가함으로써 각각의 프레임에 해당하는 `phone`을 시각화 할 수 있게 되었습니다. 

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][60:68],
        phones=ali_in[SAMPLE][60:68]
    )
)

그래프를 살펴보면 이 8 개의 프레임 안에는 세 개의 다른 phone이 있는 것으로 예측된다는 것을 확인할 수 있습니다. 

범례에서 각각의 범주를 클릭하여 그래프에서 해당 선을 on/off 할 수 있습니다. 

## Including "average" `mfcc` information

`feature_viz.py`를 이용하면 각각의 `phone`의 **average MFCC** 값을 구할 수도 있습니다. 

먼저 우리는 각각의 phone의 `MFCC`값을 모아 둔 하나의 `<dict>`를 구성해아합니다. 

In [ ]:
grouped_dict = fv.get_grouped_phones_dict(
                    feats_dict=feats_in, 
                    ali_dict=ali_in
)
list(grouped_dict.keys())[:5]

`<dict>`의 `value`는 사용한 데이터에서 해당 phone의 모든 `MFCC` 값을 담은 `<list>` 입니다. `F_B`를 예로 들면, 우리가 사용한 데이터에는 총 5,955개의 `F_B`가 있습니다. 

In [ ]:
len(grouped_dict["F_B"])

`get_average_mfccs()` 명령어를 이용하여서 각각의 phone의 `mean MFCC` 값을 구할 수 있습니다. 아래는 데이터에 나타난 모든 `F_B`의 평균 `MFCC` 벡터값입니다. 

In [ ]:
ave_dict = fv.get_average_mfccs(
    phones_dict=grouped_dict
)
ave_dict["F_B"]

`plot_frames()` 명령어를 이용하여 각각의 프레임에서 평균 벡터값을 시각화 할 수 있습니다. 

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][60:68],
        phones=ali_in[SAMPLE][60:68],
        average_mfccs_dict=ave_dict
    )
)

이제 특정 phone의 `MFCC` 값을 해당 phone의 `average MFCC` 값과 비교할 수 있습니다. `bar` 그래프 형태로 출력하면 조금 더 직관적인 비교가 가능합니다. 

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][60:68],
        phones=ali_in[SAMPLE][60:68],
        average_mfccs_dict=ave_dict,
        mode='bar'
    )
)

## Case Study of `IH`


특정 `phone`을 자세히 살펴 보면서 `MFCC`를 조금 더 이해해 봅시다. 선택한 음성 파일에는 다섯 개의 `IH` phone이 있습니다. 

In [ ]:
cat raw_data/librispeech-transcripts.txt | grep -E "1272-128104-0000"

해당하는 프레임을 찾아봅시다. 

**Note:** 해당 phone은 `IH1` 입니다. `IH0`과 `IH2`도 phone set에 존재합니다. 

In [ ]:
cat data/lang/phones.txt | grep IH

In [ ]:
enumerated_phones = list(enumerate(ali_in[SAMPLE]))

list(filter(
    lambda x: "IH" in x[1],
    enumerated_phones)
    )

우리가 찾는 다섯 개의 `IH`는 다음과 같은 프레임에 걸쳐서 나타난다는 것을 확인할 수 있습니다. 
    - 59-62
    - 96-98
    - 125-131
    - 243-245
    - 470-473

In [ ]:
ex_1 = {
    "start": 59,
    "stop": 63    # +1 to be inclusive
}

ex_2 = {
    "start": 96,
    "stop": 99    # +1 to be inclusive
}

ex_3 = {
    "start": 125,
    "stop": 132    # +1 to be inclusive
}

ex_4 = {
    "start": 243,
    "stop": 246    # +1 to be inclusive
}

ex_5 = {
    "start": 470,
    "stop": 474    # +1 to be inclusive
}

In [ ]:
ali_in[SAMPLE][ex_1["start"]:ex_1["stop"]]

각각을 시각화 해보도록 하겠습니다. 

### `ex_1`: `IH` in "MISTER"

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][ex_1["start"]:ex_1["stop"]],
        phones=ali_in[SAMPLE][ex_1["start"]:ex_1["stop"]],
        average_mfccs_dict=ave_dict
    )
)

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][ex_1["start"]:ex_1["stop"]],
        phones=ali_in[SAMPLE][ex_1["start"]:ex_1["stop"]],
        average_mfccs_dict=ave_dict,
        mode='bar'
    )
)

### `ex_2`: `IH` in "QUILTER"

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][ex_2["start"]:ex_2["stop"]],
        phones=ali_in[SAMPLE][ex_2["start"]:ex_2["stop"]],
        average_mfccs_dict=ave_dict
    )
)

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][ex_2["start"]:ex_2["stop"]],
        phones=ali_in[SAMPLE][ex_2["start"]:ex_2["stop"]],
        average_mfccs_dict=ave_dict,
        mode='bar'
    )
)

### `ex_3`: `IH` in "IS"

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][ex_3["start"]:ex_3["stop"]],
        phones=ali_in[SAMPLE][ex_3["start"]:ex_3["stop"]],
        average_mfccs_dict=ave_dict
    )
)

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][ex_3["start"]:ex_3["stop"]],
        phones=ali_in[SAMPLE][ex_3["start"]:ex_3["stop"]],
        average_mfccs_dict=ave_dict,
        mode='bar'
    )
)

### `ex_4`: `IH` in "MIDDLE"

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][ex_4["start"]:ex_4["stop"]],
        phones=ali_in[SAMPLE][ex_4["start"]:ex_4["stop"]],
        average_mfccs_dict=ave_dict
    )
)

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][ex_4["start"]:ex_4["stop"]],
        phones=ali_in[SAMPLE][ex_4["start"]:ex_4["stop"]],
        average_mfccs_dict=ave_dict,
        mode='bar'
    )
)

### `ex_5`: `IH` in "HIS"

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][ex_5["start"]:ex_5["stop"]],
        phones=ali_in[SAMPLE][ex_5["start"]:ex_5["stop"]],
        average_mfccs_dict=ave_dict
    )
)

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][ex_4["start"]:ex_4["stop"]],
        phones=ali_in[SAMPLE][ex_4["start"]:ex_4["stop"]],
        average_mfccs_dict=ave_dict,
        mode='bar'
    )
)

### `ex_3` and `ex_5`

`ex_3`은 "IS" 에서 가져온 부분이며, `ex_5`는 "HIS" 에서 가져온 부분입니다. 한 번 비교해 보겠습니다. 

In [ ]:
ex_3_5_frames = np.vstack(
    (
        feats_in[SAMPLE][ex_3["start"]:ex_3["stop"]],
        feats_in[SAMPLE][ex_5["start"]:ex_5["stop"]]
    )
)
ex_3_5_frames.shape

In [ ]:
ex_3_5_phones = ali_in[SAMPLE][ex_3["start"]:ex_3["stop"]] + ali_in[SAMPLE][ex_5["start"]:ex_5["stop"]]
ex_3_5_phones

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=ex_3_5_frames,
        phones=ex_3_5_phones,
        average_mfccs_dict=ave_dict,
        mode='bar'
    )
)

`IH`를 들여다보면 각각의 프레임에서 유사점을 찾기는 어렵습니다. 하지만 평균을 내어보면 `IH_B`와 `IH_I`는 유사한 점이 있다는 것을 확인할 수 있습니다. 

하지만 여전히 왜 개별 프레임은 너무나도 다른지에 대한 의문이 남습니다. 

### phones in context

이는 각각의 `IH`가 서로 다른 환경에서 발화되었기 때문이라고 볼 수 있습니다. 즉, 주변에 위치한 phone이 다르기 때문에 `MFCC`에 다른 영향을 끼쳤다고 볼 수 있습니다. (*i.e.,* `coarticulation`)

#### `ex_1` ("MIS..")

`ex_1`("MISTER")을 다시 시각화 해 봅시다. 이번에는 해당 phone의 앞 뒤에 몇 개의 프레임을 같이 출력해 보겠습니다. 

In [ ]:
py.iplot(
    fv.plot_frames(
        frames=feats_in[SAMPLE][ex_1["start"]-2:ex_1["stop"]+2],
        phones=ali_in[SAMPLE][ex_1["start"]-2:ex_1["stop"]+2],
        average_mfccs_dict=ave_dict
    )
)

`3/8` 프레임과 `4/8` 프레임(`IH`의 첫 두 프레임)에서 유사점을 발견할 수 있습니다. 

`3/8` 프레임과 `6/8` 프레임(`IH`의 *시작*과 *끝* 프레임)을 살펴보면 매우 다르다는 것을 볼 수 있습니다. 

`2/8` 프레임과 `3/8` 프레임을 비교해보고 (`M` 의 마지막 프레임과 `IH`의 첫 프레임) `7/9` 프레임과 `8/9` 프레임(`IH`의 마지막 프레임과 `S`의 첫 프레임)을 비교해 보겠습니다. 